In [0]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
import spacy
from spacy.lang.en import English
from sklearn.ensemble import RandomForestClassifier
from collections import Counter

In [0]:
#!pip install spacy
#!apt install -qq enchant
#!pip install pyenchant
#!python -m spacy download en

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

Import Data

In [0]:
data = pd.read_csv('https://raw.githubusercontent.com/XaviJunior/SBB/master/project_2/Data/train.csv',encoding="utf8")

In [0]:
y=data['target']
X=data['text']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=14)

In [0]:
Text_train=X_train.values.tolist()
Text_test=X_test.values.tolist()

Data Cleaning

In [0]:
#Tokenizer found in the DMML class lab 6.1
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [0]:
vect=TfidfVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,1))

In [0]:
Clean_train=vect.transform(Text_train)
Clean_test=vect.transform(Text_test)

In [0]:
data_test=pd.read_csv('https://raw.githubusercontent.com/XaviJunior/SBB/master/project_2/Data/test.csv',encoding='utf8')
D=data_test['text']
d=vect.fit_transform(D)

ML

In [0]:
RF = RandomForestClassifier(criterion="entropy", n_estimators=51, max_depth=306, random_state=8)
RF.fit(Clean_train,y_train)
RF.score(Clean_test,y_test)

First Prediction


In [0]:
pred=RF.predict(d)
data_test['target']=pred
Fin=data_test[['id','target']]

In [0]:
from google.colab import drive
drive.mount('drive')
Fin.to_csv('UNIL_SBB_1st.csv',index=False)
!cp UNIL_SBB_1st.csv "drive/My Drive/"

Result = 0.77811
Rank=1763

Random Forest Optim

In [0]:
a=0
for i in range(11,500,2):
  for j in range(50,200):
    RF_2 = RandomForestClassifier(criterion="entropy", n_estimators=i, max_depth=j, random_state=8)
    RF_2.fit(Clean_train,y_train)
    if RF_2.score(Clean_test,y_test)>a:
      a=RF_2.score(Clean_test,y_test)
      print(a,' ',i,' ',j)

0.7393302692055155   11   50
0.7498358502954695   11   51
0.7511490479317138   11   53
0.7518056467498359   11   57
0.7550886408404465   11   58
0.7564018384766907   11   64
0.757715036112935   11   69
0.762967826657912   11   74
0.7655942219304005   11   80
0.767564018384767   11   86
0.7688772160210111   11   92
0.7695338148391333   11   110
0.7701904136572554   11   116
0.7747866053841103   11   119
0.7800393959290873   11   132
0.7806959947472094   11   144
0.7826657912015759   11   152
0.783322390019698   11   165
0.7839789888378201   11   170
0.7846355876559422   11   179
0.7852921864740644   11   183
0.7898883782009193   11   197
0.7918581746552856   13   170
0.7951411687458962   15   170
0.7957977675640184   23   176
0.7971109652002626   25   186


In [0]:
import xgboost as xgb

In [0]:
a=0
e=0
d=0
for i in range(103,500,2):
  for j in range(10,500):
    xgc = xgb.XGBClassifier(n_estimators=i, max_depth=j, base_score=0.5, objective='binary:logistic', random_state=42)
    xgc.fit(Clean_train,y_train)
    if xgc.score(Clean_test,y_test)>a:
      a=xgc.score(Clean_test,y_test)
      print(a,' ',i,' ',j)

In [0]:
0.7957977675640184   101   71
0.7806959947472094   101   60

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB

In [32]:
from google.colab import drive
drive.mount('drive')
Fin.to_csv('UNIL_SBB_1st.csv',index=False)
!cp UNIL_SBB_1st.csv "drive/My Drive/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [0]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
('vect', vect),
#('tfidf', TfidfTransformer()),
('clf',MultinomialNB()),])

In [27]:
text_clf.fit(Text_train, y_train)
print(text_clf.score(Text_test,y_test))

0.8102429415627052
